# Options Data

---

In [18]:
# importing libraries
from datetime import datetime, timezone, timedelta, time
import pandas as pd
import numpy as np
import csv
import time as tm
import re
import json
from tqdm import tqdm
import logging
import timeit
import concurrent.futures


In [37]:
year = 2020
month = ''

In [20]:
df = pd.read_csv(f'data/{year}/NiftySpotData-{month}{year}-1min-Signal-Expiry.csv')
# df = pd.read_csv(f'/home/algolinux/7/D$/HirakDrive/workspace/GitHub/Endovia/data/{year}/NiftySpotData-{year}-1min-Signal-Expiry.csv')

df["Datetime"] = pd.to_datetime(df["Datetime"], format='%Y-%m-%d %H:%M:%S%z')
df['closest_expiry'] = pd.to_datetime(df['closest_expiry'], format='%Y-%m-%d %H:%M:%S%z')

# # Convert 'Datetime' to datetime and add timezone information '+05:30'
# df['Datetime'] = pd.to_datetime(df['Datetime'], format='%Y-%m-%d %H:%M:%S')
# df['Datetime'] = df['Datetime'].dt.strftime('%Y-%m-%d %H:%M:%S') + '+05:30'

# # Convert 'closest_expiry' to datetime and add time component '00:00:00+00:00'
# df['closest_expiry'] = pd.to_datetime(df['closest_expiry'], format='%Y-%m-%d')
# df['closest_expiry'] = df['closest_expiry'].dt.strftime('%Y-%m-%d') + ' 00:00:00+00:00'

# # Convert 'Datetime' back to datetime with timezone information
# df['Datetime'] = pd.to_datetime(df['Datetime'], format='%Y-%m-%d %H:%M:%S%z')

# # Convert 'closest_expiry' back to datetime with timezone information
# df['closest_expiry'] = pd.to_datetime(df['closest_expiry'], format='%Y-%m-%d %H:%M:%S%z')


try:
    # df drop PL PH, PL_val, PH_val, slope_ph, slope_pl, upper, lower, upos, dnos, IsPivotHigh
    df.drop(['PL', 'PH', 'PL_val', 'PH_val', 'slope_ph', 'slope_pl', 'upper', 'lower', 'upos', 'dnos', 'IsPivotHigh', 'Slope'], axis=1, inplace=True)
    df.drop(['Unnamed: 0'], axis=1, inplace=True)
except:
    pass

# Initialize all columns with NaN
df = df.assign(
    position='', atmSP=np.nan, wingCall=np.nan, wingPut=np.nan, legPriceOrignal1=np.nan, legPriceOrignal2=np.nan,
    legPriceOrignal3=np.nan, legPriceOrignal4=np.nan, lpos1=np.nan, lpos2=np.nan, lpos3=np.nan, lpos4=np.nan,
    legAfterPos1=np.nan, legAfterPos2=np.nan, legAfterPos3=np.nan, legAfterPos4=np.nan, legAfterPosDiff1=np.nan,
    legAfterPosDiff2=np.nan, legAfterPosDiff3=np.nan, legAfterPosDiff4=np.nan, legPriceFinal1=np.nan,
    legPriceFinal2=np.nan, legPriceFinal3=np.nan, legPriceFinal4=np.nan, m2m1=np.nan, m2m2=np.nan, m2m3=np.nan,
    m2m4=np.nan, totalPL=0, cumReturns=0, balance=0
)

# only keep values between 9:15 to 15:30
df = df[(df['Datetime'].dt.time >= time(9, 15)) & (df['Datetime'].dt.time <= time(15, 30))]

# drop rows with same Datetime
df.drop_duplicates(subset='Datetime', keep='first', inplace=True)

# reset df index
df.reset_index(drop=True, inplace=True)


C:\Users\ALGO1\AppData\Local\Temp\2\ipykernel_14968\410421955.py:1: DtypeWarning: Columns (8,9,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'data/{year}/NiftySpotData-{month}{year}-1min-Signal-Expiry.csv')


In [21]:
# account for missing optionsData by dropping the rows from df where Datetime is
# dates_to_drop = ['2023-02-16', '2023-03-07', '2023-03-30', '2023-04-04', '2023-09-19', '2023-04-25', '2023-01-20', '2023-10-22', '2023-10-24']
# dates_to_drop = ['2022-09-22']
# dates_to_drop = ['2021-01-08', '2021-02-24', '2021-07-16', '2021-08-02', '2021-11-03', '2021-11-04']
# dates_to_drop = ['2020-01-31', '2020-03-13']
dates_to_drop = ['2024-03-02']
for date in dates_to_drop:
    df = df[df['Datetime'].dt.date != pd.to_datetime(date).date()]

df.reset_index(drop=True, inplace=True)

,High,Low,Open,Close,Datetime,ama,rsi,atr,signal,closest_expiry,...,legPriceFinal2,legPriceFinal3,legPriceFinal4,m2m1,m2m2,m2m3,m2m4,totalPL,cumReturns,balance
0,17478.10,17387.15,17387.15,17462.35,2022-01-03 09:15:00+05:30,NaN,NaN,90.95,Hold,2022-01-05 00:00:00+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
1,17451.45,17428.40,17428.40,17450.55,2022-01-03 09:16:00+05:30,NaN,NaN,90.95,Hold,2022-01-05 00:00:00+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
2,17459.55,17442.05,17450.05,17457.70,2022-01-03 09:17:00+05:30,NaN,NaN,90.95,Hold,2022-01-05 00:00:00+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
3,17465.75,17455.30,17455.95,17464.55,2022-01-03 09:18:00+05:30,NaN,NaN,90.95,Hold,2022-01-05 00:00:00+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
4,17474.65,17462.15,17465.25,17473.20,2022-01-03 09:19:00+05:30,NaN,NaN,90.95,Hold,2022-01-05 00:00:00+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0


In [35]:
# optionsData = pd.read_csv(f'data/{year}/NiftyOptionsData-{month}{year}-1min.csv')
# optionsData.rename(columns={'Expirydate': 'ExpiryDate'}, inplace=True)

# #make a new column named Instrument Type 
# optionsData['InstrumentType'] = optionsData['Ticker'].apply(lambda x: 'CE' if x[-6] == 'C' else 'PE') 

# #save as csv file with same name
# optionsData.to_csv(f'data/{year}/NiftyOptionsData-{month}{year}-1min.csv', index=False)

# optionsData

In [36]:
# #print unique Ticker values
# set(optionsData['Ticker'].unique())


In [27]:
optionsData = pd.read_csv(f'data/{year}/NiftyOptionsData-{month}{year}-1min.csv')
# optionsData = pd.read_csv(f'/home/algolinux/7/D$/HirakDrive/workspace/GitHub/Endovia/data/{year}/NiftyOptionsData-{year}-1min.csv')
optionsData['Datetime'] = pd.to_datetime(optionsData['Datetime'], format='%Y-%m-%d %H:%M:%S')
# optionsData['Datetime'] = optionsData['Datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
# optionsData['Datetime'] = pd.to_datetime(optionsData['Datetime'], format='%Y-%m-%d %H:%M:%S')
optionsData['ExpiryDate'] = pd.to_datetime(optionsData['ExpiryDate'], format='%Y-%m-%d %H:%M:%S%z')
optionsData['ExpiryDate'] = optionsData['ExpiryDate'].dt.strftime('%Y-%m-%d %H:%M:%S%z')
optionsData['ExpiryDate'] = pd.to_datetime(optionsData['ExpiryDate'], format='%Y-%m-%d %H:%M:%S%z')



optionsData = optionsData.sort_values(by='Datetime')
optionsData.reset_index(drop=True, inplace=True)

KeyError: 'ExpiryDate'

In [ ]:
# optionsData = pd.read_csv(f'data/{year}/NiftyOptionsData-May{year}-1min.csv')

# # Drop unnecessary column if exists
# if 'Unnamed: 0' in optionsData.columns:
#     optionsData.drop(['Unnamed: 0'], axis=1, inplace=True)

# # Convert the 'Datetime' column to datetime format with timezone information
# optionsData['Datetime'] = pd.to_datetime(optionsData['Datetime'], format='%Y-%m-%d %H:%M:%S%z')

# # Remove timezone information for filtering purposes (convert to naive datetime)
# optionsData['Datetime'] = optionsData['Datetime'].dt.tz_localize(None)

# # Convert the 'Expiry Date' column to datetime with timezone information
# optionsData['Expiry Date'] = pd.to_datetime(optionsData['Expiry Date'], format='%Y-%m-%d %H:%M:%S %z')

# # Set the time component to '00:00:00' and timezone to '+00:00'
# optionsData['Expiry Date'] = optionsData['Expiry Date'].dt.tz_convert('UTC').dt.normalize()

# # Rename 'Strike Price' to 'Strike'
# optionsData.rename(columns={'Strike Price': 'Strike'}, inplace=True)

# # Ensure 'Datetime' is in datetime format
# optionsData['Datetime'] = pd.to_datetime(optionsData['Datetime'], errors='coerce')

# # Keep only data where time is ending in 00, 15, 30, 45 minutes
# optionsData = optionsData[optionsData['Datetime'].dt.minute % 15 == 0]

# # Sort the data by 'Datetime' and reset the index
# optionsData = optionsData.sort_values(by='Datetime')
# optionsData.reset_index(drop=True, inplace=True)

# # Save the cleaned data back to a CSV file
# optionsData.to_csv(f'data/{year}/NiftyOptionsData-May{year}-1min.csv', index=False)


In [ ]:
optionsData.head()

,Datetime,Open,High,Low,Close,Ticker,ExpiryDate,Strike,Instrument Type
0,2022-01-03 09:15:00,135.50,135.50,135.50,135.50,NIFTY03FEB2216050PE.NFO,2022-02-03 00:00:00+00:00,16050.0,PE
1,2022-01-03 09:15:00,1.70,1.70,1.40,1.45,NIFTY06JAN2215500PE.NFO,2022-01-06 00:00:00+00:00,15500.0,PE
2,2022-01-03 09:15:00,55.20,57.00,53.90,54.75,NIFTY13JAN2217750CE.NFO,2022-01-13 00:00:00+00:00,17750.0,CE
3,2022-01-03 09:15:00,1.45,1.45,0.95,1.20,NIFTY06JAN2215100PE.NFO,2022-01-06 00:00:00+00:00,15100.0,PE
4,2022-01-03 09:15:00,1.05,1.70,1.05,1.15,NIFTY06JAN2215150PE.NFO,2022-01-06 00:00:00+00:00,15150.0,PE


In [ ]:
# updated get close price fucntion
def get_close_price(strike_price, option_type, current_datetime, expiry_date):
    try:
        # First filter based on the first criteria
        
        # filtered_opd = optionsData.loc[(optionsData['Datetime'] == current_datetime) & (optionsData['Strike'] == strike_price) & (optionsData['ExpiryDate'] == expiry_date) & (optionsData['Instrument Type'] == option_type)]
        
        expiry_date = pd.to_datetime(expiry_date).date()
        filtered_opd = optionsData[optionsData['Datetime'] == current_datetime]
        filtered_opd = filtered_opd[filtered_opd['Instrument Type'] == option_type]
        filtered_opd = filtered_opd[filtered_opd['Strike'] == strike_price]
        filtered_opd['ExpiryDate'] = pd.to_datetime(filtered_opd['ExpiryDate']).dt.date
        filtered_opd = filtered_opd[filtered_opd['ExpiryDate'] == expiry_date]
        
        

        # Now, apply the second criteria on the filtered DataFrame
        result = filtered_opd['Close']

        if result.empty:
            logging.warning(f"No close price found for {strike_price}, {option_type} at {current_datetime} for expiry {expiry_date}")
            return 0
        else:
            return result.values[0]
    except Exception as e:
        logging.error(f"Error while fetching close price for {strike_price} at {current_datetime}")
        logging.error(e)



# get close price dict
def get_close_price_dict(atm_sp, wing_call, wing_put, current_datetime, expiry_date, i, df):
    try:
        # Define a list to store the futures
        futures = []

        # Create a ThreadPoolExecutor with a maximum of 4 threads
        with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
            # Submit tasks to fetch close prices asynchronously
            futures.append(executor.submit(get_close_price, atm_sp, 'CE', current_datetime, expiry_date))
            futures.append(executor.submit(get_close_price, atm_sp, 'PE', current_datetime, expiry_date))
            futures.append(executor.submit(get_close_price, wing_call, 'CE', current_datetime, expiry_date))
            futures.append(executor.submit(get_close_price, wing_put, 'PE', current_datetime, expiry_date))

        # Retrieve the results
        results = [future.result() for future in futures]
        
        try:
            for j in range(len(results)):
                if results[j] == False:
                    previous_value = df.at[i - 1, f'legPriceOrignal{j + 1}']
                    if np.isnan(previous_value):
                        results[j] = 0
                    else:
                        results[j] = previous_value
        except Exception as e:
            print(f"Error in get_close_price_dict: {e}")

        # Map the results to the ticker names
        close_prices = {
            'atmSPCall': results[0],
            'atmSPPut': results[1],
            'wingCallPrice': results[2],
            'wingPutPrice': results[3]
        }
        
        
        return close_prices, df
    except Exception as e:
        logging.error(f"Error in get_close_price_dict: {e}")

In [ ]:
# positions
positions = {'beginx': [-1, -1, 1, 1], 'buy': [0, -2, 2, 2], 'sell': [-2, 0, 2, 2], 'squareoff': [0, 0, 0, 0], 'hard-squareoff': [0, 0, 0, 0]}

In [ ]:
# Function to check Stop Loss
def check_stoploss(current_balance, capital_at_start_of_trade):
    # Calculate the difference in capital from the start to the current balance
    difference_in_capital = capital_at_start_of_trade - current_balance

    # Check if the difference is more than 3% of the initial capital
    return difference_in_capital > 0.03 * capital_at_start_of_trade

In [ ]:
# m2m calculation
num_legs = 4

columns_to_reset = [ 'atmSP', 'wingCall', 'wingPut', 'legPriceOrignal1', 'legPriceOrignal2',
    'legPriceOrignal3', 'legPriceOrignal4', 'balance', 'lpos1', 'lpos2', 'lpos3',
    'lpos4', 'legAfterPos1', 'legAfterPos2', 'legAfterPos3', 'legAfterPos4',
    'legAfterPosDiff1', 'legAfterPosDiff2', 'legAfterPosDiff3', 'legAfterPosDiff4',
    'legPriceFinal1', 'legPriceFinal2', 'legPriceFinal3', 'legPriceFinal4',
    'm2m1', 'm2m2', 'm2m3', 'm2m4', 'totalPL', 'cumReturns'
]

lpos_column_names = [f"lpos{i+1}" for i in range(num_legs)]


def calculate_m2m_new(index, start_day=False, caller='o'):
    if str(df.at[index, 'position']).lower() == 'hold':
        df.at[index, 'totalPL'] = 0
        df.at[index, 'cumReturns'] = df.at[index-1, 'cumReturns']
        df.at[index, 'balance'] = df.at[index-1, 'balance']
        return
    position_to_take = positions[str(df.at[index, 'position'])]
    
    
    for i in range(0, num_legs):
        df.at[index, f"lpos{i+1}"] = position_to_take[i]
        # Calculating the leg after position
        df.at[index, f"legAfterPos{i+1}"] = df.at[index, f"legPriceOrignal{i+1}"] * df.at[index, f"lpos{i+1}"]
        

        # Calculating the difference between the leg after position and the previous leg after position
        difference_in_position = df.at[index, f"lpos{i+1}"] - df.at[index-1, f"lpos{i+1}"]
        df.at[index, f"legAfterPosDiff{i+1}"] = difference_in_position * df.at[index, f"legPriceOrignal{i+1}"]

        # Calculating Final Leg Price
        df.at[index, f"legPriceFinal{i+1}"] = df.at[index, f"legAfterPosDiff{i+1}"] + df.at[index-1, f"legAfterPos{i+1}"]

        if not start_day:
            # Calculating m2m
            df.at[index, f"m2m{i+1}"] = df.at[index, f"legPriceFinal{i+1}"] - df.at[index, f"legAfterPos{i+1}"]
        else:
            df.at[index, f"m2m{i+1}"] = 0

        
        # calculating the current data pnl which is the sum of all m2ms
        df.at[index, 'totalPL'] += df.at[index, f"m2m{i+1}"]

    # multiplying the totalPL by 50 for the lot size
    df.at[index, 'totalPL'] = df.at[index, 'totalPL'] * -1 * 50
    df.at[index, 'cumReturns'] = df.at[index-1, 'cumReturns'] + df.at[index, 'totalPL'] 
    # updating the balance
    df.at[index, 'balance'] = df.at[index-1, 'balance'] + df.at[index, 'totalPL']

In [ ]:
squareoff_columns_reset = ['balance', 'lpos1', 'lpos2', 'lpos3',
    'lpos4', 'legAfterPos1', 'legAfterPos2', 'legAfterPos3', 'legAfterPos4',
    'legAfterPosDiff1', 'legAfterPosDiff2', 'legAfterPosDiff3', 'legAfterPosDiff4',
    'legPriceFinal1', 'legPriceFinal2', 'legPriceFinal3', 'legPriceFinal4',
    'm2m1', 'm2m2', 'm2m3', 'm2m4', 'totalPL', 'cumReturns']
    
# round to nearest 50
def round_to_nearest_50(number):
    return round(number / 50) * 50
    
for i in range(1, len(df)):
    df.at[df.index[i], 'atmSP'] = round_to_nearest_50(df.at[df.index[i], 'Close'])

In [ ]:
# Function to determine if the current time is before the target time

def is_before_target_time(current_datetime, target_time="15:10:00+05:30"):
    target_time = datetime.strptime(target_time, "%H:%M:%S%z")
    # Compare the time parts
    return current_datetime.time() < target_time.time()


def get_daily_diff(current_datetime):
    day_of_week = current_datetime.weekday()
    # Define a dictionary to map each day to its corresponding difference
    day_diff_mapping = {
        "Monday": 400,
        "Tuesday": 300,
        "Wednesday": 200,
        "Thursday": 100,
        "Friday": 500,
        "Saturday": 0,
        "Sunday": 0
    }
    # Return the difference based on the day of the week
    return day_diff_mapping[datetime.strftime(current_datetime, '%A')]

In [ ]:
# function to save data to csv
def save_data():
    df.to_csv(f'data/backtest/backtest{month}{year}-adaptiveSL-1min.csv')
    # df.to_csv(f'/home/algolinux/7/D$/HirakDrive/workspace/GitHub/Endovia/data/backtest/backtest{year}-adaptiveSL.csv')



In [ ]:
logging.basicConfig(filename='logs.log', filemode='w', encoding='utf-8', level=logging.DEBUG, force=True)


In [ ]:
position = "squareoff"

# set the initial capital and the current capital hi hirak
capital = 100000

logging.info("*"*100)

first_run = True
highest_m2m = 0

for i, row in enumerate(df.itertuples()):
    # timer_start = tm.time()
    # Check if this a new day and time is 09:30:00+05:30

    current_datetime_str = str(row.Datetime)[:-6]
    current_datetime_str_short = current_datetime_str

    has_traded_today = False
    has_squareoff_today = False
    logging.info(f"Current Datetime: {current_datetime_str}")

    # check if the time is 09:30:00+05:30 or it is the first run
    if '09:45:00' in current_datetime_str or first_run:
        highest_m2m = 0
        # Calculate the ATM strike price
        atmSP = round_to_nearest_50(row.Close)
        wingPut = atmSP - get_daily_diff(row.Datetime)
        wingCall = atmSP + get_daily_diff(row.Datetime)
        
        # marking the postion as 0 updating tickers_dict
        close_price_dict, df = get_close_price_dict(atmSP, wingCall, wingPut, current_datetime_str_short, row.closest_expiry, i, df)

        df.loc[i, ['atmSP', 'wingCall', 'wingPut', 'legPriceOrignal1', 'legPriceOrignal2', 'legPriceOrignal3', 'legPriceOrignal4']] = [atmSP, wingCall, wingPut, close_price_dict['atmSPCall'], close_price_dict['atmSPPut'], close_price_dict['wingCallPrice'], close_price_dict['wingPutPrice']]
        if first_run:
            # Reset values to 0 for the specified columns at row i
            df.loc[i, columns_to_reset] = 0
            df.at[i, 'position'] = 'hold'
            position = 'hold'
            df.at[i, 'balance'] = capital
            first_run = False
        else:
            df.at[i, 'position'] = 'beginx'
            position = 'hold'
            calculate_m2m_new(i, start_day=True, caller='beginx 9:45')
            df.at[i, 'stoploss'] = highest_m2m - (100000*0.03)
        # sell atmSP call -> api call # sell atmSp put -> api call # buy wingPut -> api call # buy wingCall -> api call
    
    elif df.at[i-1, 'position'] == 'hard-squareoff' and row.Datetime.time() > time(9, 30):
        # if previous was hard-squareoff then hust continue and forward the values
        close_price_dict, df = get_close_price_dict(atmSP, wingCall, wingPut, current_datetime_str_short, row.closest_expiry, i, df)
        df.loc[i, ['atmSP', 'wingCall', 'wingPut', 'legPriceOrignal1', 'legPriceOrignal2', 'legPriceOrignal3', 'legPriceOrignal4']] = [atmSP, wingCall, wingPut, close_price_dict['atmSPCall'], close_price_dict['atmSPPut'], close_price_dict['wingCallPrice'], close_price_dict['wingPutPrice']]
        df.at[i, 'position'] = position = 'hard-squareoff'
        df.at[i, 'balance'] = df.at[i-1, 'balance']
        df.at[i, 'cumReturns'] = df.at[i-1, 'cumReturns']
        df.at[i, 'totalPL'] = 0
    
    # if time is 15:15:00+05:30 then squareoff all positions
    elif row.Datetime.time() == time(15, 15):
        # buy atmSp call -> api call # buy atmSp put -> api call # sell wingPut -> api call # sell wingCall -> api call
        capital = df.at[i-1, 'balance']
        df.at[i, 'balance'] = df.at[i-1, 'balance']
        df.at[i, 'position'] = position = 'squareoff'
        # store the values in the dataframe
        close_price_dict, df = get_close_price_dict(atmSP, wingCall, wingPut, current_datetime_str_short, row.closest_expiry, i, df)
        df.loc[i, ['atmSP', 'wingCall', 'wingPut', 'legPriceOrignal1', 'legPriceOrignal2', 'legPriceOrignal3', 'legPriceOrignal4']] = [atmSP, wingCall, wingPut, close_price_dict['atmSPCall'], close_price_dict['atmSPPut'], close_price_dict['wingCallPrice'], close_price_dict['wingPutPrice']]         
        calculate_m2m_new(i, caller='15:15')
    
    elif row.Datetime.time() > time(15, 15) and row.Datetime.time() <= time(15, 30):
        # store the values in the dataframe
        df.loc[i, ['atmSP', 'wingCall', 'wingPut'] ] = [atmSP, wingCall, wingPut]
        df.at[df.index[i], 'position'] = position =  'hold'
        df.at[i, 'balance'] = df.at[i-1, 'balance']
    
    # between 9:15:00+05:30 and 9:29:00+05:30
    elif row.Datetime.time() < time(9, 45) and row.Datetime.time() >= time(9, 15):
        # df.loc[i, ['atmSP', 'wingCall', 'wingPut'] ] = [atmSP, wingCall, wingPut]
        df.loc[i, ['atmSP', 'wingCall', 'wingPut'] ] = [atmSP, wingCall, wingPut]
        df.at[df.index[i], 'position'] = 'hold'
        logging.debug(f"balance at 9:15: {df.at[i-1, 'balance']}")
        df.at[i, 'balance'] = df.at[i-1, 'balance']
        position = 'hold'
    
    elif df.at[i-1, 'position'] == 'squareoff' and row.Datetime.time() > time(9, 30):

        # check if current Datetime minutes is a 00, 15, 30, 45
        if row.Datetime.minute % 15 == 0:
            # Calculate the ATM strike price
            atmSP = round_to_nearest_50(row.Close)
            wingPut = atmSP - get_daily_diff(row.Datetime)
            wingCall = atmSP + get_daily_diff(row.Datetime)
            close_price_dict, df = get_close_price_dict(atmSP, wingCall, wingPut, current_datetime_str_short, row.closest_expiry, i, df)
            df.at[i, 'position'] = position = 'beginx'
            df.loc[i, ['atmSP', 'wingCall', 'wingPut', 'legPriceOrignal1', 'legPriceOrignal2', 'legPriceOrignal3', 'legPriceOrignal4']] = [atmSP, wingCall, wingPut, close_price_dict['atmSPCall'], close_price_dict['atmSPPut'], close_price_dict['wingCallPrice'], close_price_dict['wingPutPrice']]
            calculate_m2m_new(i, start_day=True, caller='beginx')
        else:
            df.at[i, 'position'] = position = 'squareoff'
            df.at[i, 'balance'] = df.at[i-1, 'balance']
            df.at[i, 'cumReturns'] = df.at[i-1, 'cumReturns']
            df.at[i, 'totalPL'] = 0
        
    
    else:
        close_price_dict, df = get_close_price_dict(atmSP, wingCall, wingPut, current_datetime_str_short, row.closest_expiry, i, df)
        if position == 'hold':
            if row.signal == 'Buy':
                if row.rsi < 70 and row.ama < row.Close:
                    # buy atmSp call -> api call # Sell atmSp put -> api call # buy wingCall -> api call # buy wingPut -> api call
                    df.at[i, 'position'] = position = 'buy'
                    has_traded_today = True
            
            elif row.signal == 'Sell':
                if row.rsi > 30 and row.ama > row.Close:
                    # sell atmSp call -> api call # buy atmSp put -> api call # Buy wingPut -> api call # Buy wingCall -> api call
                    df.at[i, 'position'] = position = 'sell'
                    has_traded_today = True

        elif position == 'buy':
            if row.signal == 'Hold':
                if row.rsi > 70:
                    # buy atmSp put -> api call x 2 # sell wingPut -> api call x 2 # sell wingCall -> api call x 2
                    df.at[i, 'position'] = position = 'squareoff'
                    has_traded_today = True
            
            elif row.signal == 'Sell':          
                # buy atmSp put -> api call # sell wingPut -> api call # sell wingCall -> api call
                df.at[i, 'position'] = position =  'squareoff'
                has_traded_today = True

        elif position == 'sell':
            if row.signal == 'Hold':
                if row.rsi < 30:              
                    # buy atmSp call -> api call # sell wingPut -> api call # sell wingCall -> api call  
                    df.at[i, 'position'] = position = 'squareoff'
                    has_traded_today = True

                    
            elif row.signal == 'Buy':
                df.at[df.index[i], 'position'] = position = 'squareoff'
                has_traded_today = True
                

        df.at[i, 'position'] = position
        df.loc[i, ['atmSP', 'wingCall', 'wingPut', 'legPriceOrignal1', 'legPriceOrignal2', 'legPriceOrignal3', 'legPriceOrignal4']] = [atmSP, wingCall, wingPut, close_price_dict['atmSPCall'], close_price_dict['atmSPPut'], close_price_dict['wingCallPrice'], close_price_dict['wingPutPrice']]


        if not has_traded_today:
            df.at[i, 'position'] = df.at[i-1, 'position']

        
        # called everytime
        calculate_m2m_new(i, caller='daily')

        if df.at[i, 'cumReturns'] > highest_m2m:
            highest_m2m = df.at[i, 'cumReturns']
        
        df.at[i, 'stoploss'] = stoploss_price = highest_m2m - (100000*0.03)

        if df.at[i, 'cumReturns'] < stoploss_price:
            logging.info(f" Stoploss hit at {current_datetime_str} with highest m2m: {highest_m2m} and stoploss price: {stoploss_price}")
            logging.info(f" Stoploss hit at {current_datetime_str}")
            # squareoff all positions
            df.loc[i, squareoff_columns_reset] = 0
            df.at[i, 'position'] = position = 'hard-squareoff'
            calculate_m2m_new(i, caller='hard-squareoff')
            highest_m2m = df.at[i, 'cumReturns']
            

            

        
    
    # timer_end = tm.time()
    # logging.info(f"Time taken for this iteration: {timer_end - timer_start} seconds")


    # df.at[df.index[i], 'position'] = position


save_data()





Error in get_close_price_dict: -1


C:\Users\ALGO1\AppData\Local\Temp\2\ipykernel_14968\3244970346.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5.449999999999996' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'totalPL'] += df.at[index, f"m2m{i+1}"]
C:\Users\ALGO1\AppData\Local\Temp\2\ipykernel_14968\3244970346.py:49: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-14.999999999999591' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'cumReturns'] = df.at[index-1, 'cumReturns'] + df.at[index, 'totalPL']
C:\Users\ALGO1\AppData\Local\Temp\2\ipykernel_14968\3244970346.py:51: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '99987.5' has dtype incompatible with int64, plea

---
